# FARYADELL SIMULATION FRAMEWORK
This package has been provided to simulate easily and quickly several useful tools that have been written to simulate `linear` and `nonlinear dynamic` systems, dependent or independent over time. If you are a student who whishes to investigate and has decided to seek dynamic and control systems, this is for you. Enjoy it.

* Creator:   Abolfazl Delavar
* Web:       https://github.com/abolfazldelavar
* Updated:   8 April 2023

"*Successful people are those who can build solid foundations with the bricks others throw at them.*"
    
Run the below codes to observe the script timing.
* `python -m cProfile -o data\outputs\profiler\cprofiler.prof main.py`
* `snakeviz data\outputs\profiler\cprofiler.prof`

## Requirements
All external **extensions** utilized in the project must be added in this section.

In [31]:
from core.lib.pyRequirment   import *
from core.lib.coreLib        import *
from core.caller.dyEngine    import *
from core.caller.scopeEngine import *
from core.caller.estEngine   import *
from blocks.compNeuronBlocks import *
from blocks.chaosBlocks      import *
from blocks.industrialBlocks import *
from cv2 import imread, cvtColor, COLOR_BGR2GRAY

## Libraries
To have comprehensive accessibility to libraries, if there is some you would like to add, you can put them as an object to have better control and management of your libraries. Furthermore, as default, there are two libraries, `func` and `draw`, in which there are several useful functions that could be used for drawing or other purposes.

In [2]:
lib = {
    'func' : functionLib(),
    'draw' : plotToolLib(),
}

## Specialized functions
As long as there is any specific function that you have to define, here is the best place to reach this aim. This section is considered as a pre-defined function step that could be available to use in the whole project.

### Neuron synapses
Neuron synapses moedel which is obtained from an exponential distribution. The mathematical equations are given below.


\begin{equation}
f_{\text{R}}(r) = \left\{
  \begin{matrix}
    \frac{1}{\frac{1}{\lambda} e^{-r/\lambda}} & r > 0 \\ 
    0 & \text{Otherwise}
  \end{matrix}\right.
\end{equation}

\begin{equation}
  x_{\text{Post}} = \bigl \lceil {x_{\text{Pre}} + r \cos{(\phi)}} \bigr \rceil,
  y_{\text{Post}} = \bigl \lceil {y_{\text{Pre}} + r \sin{(\phi)}} \bigr \rceil
\end{equation}

The number of output synapses $N_{out}^{(i,j)}$ can be adjusted but, the number of input synapses for each neuron is generated randomly based on the predefined number of output synapses, the radius of connections ($r$), and the probability of connection ($\lambda$). The exponential distribution rate $f_\text{R}(r)$ depends on the coordinates of connections which are evaluated by calculating the sine and cosine of an angle $\left(\phi\right)$ which is uniformly distributed in the range of $[0, 2\pi]$.

In [3]:
def createNeuronsConnections(params):
    # 3D connection of neurons and synaptic connections
    Post         = np.zeros((params['mneuro'], params['nneuro'], params['N_connections']), dtype=np.int16)
    ties_stock   = 2000 * params['N_connections']
    
    for i in range(0, params['mneuro']):
        for j in range(0, params['nneuro']):
            XY      = np.zeros((2, ties_stock), dtype=np.int8)
            R       = np.random.exponential(scale=params['lambdagain'], size=ties_stock)
            fi      = 2 * np.pi * np.random.rand(1, ties_stock)
            XY[0,:] = np.int16(R * np.cos(fi))
            XY[1,:] = np.int16(R * np.sin(fi))
            _, idx  = np.unique(XY, axis=1, return_index=True)
            XY      = XY[:, np.sort(idx)] # returns the same data with no repetition
            n       = 0
            
            for k in range(0, np.size(XY, 1)):
                x = i + XY[0, k]
                y = j + XY[1, k]
                # This is distance condition, with sin and cos we applied a distance condition == 1
                # this line will evaluate that == 0
                pp = 1 if i==x and j == y else 0
                if (x>=0 and y>=0 and x<params['mneuro'] and y<params['nneuro'] and pp==0):
                    # Returns the linear index equivalents to the rows(x) and columns(y) (based on MATLAB)
                    Post[i,j,n] = y*params['mneuro'] + x
                    n += 1
                if n >= params['N_connections']: break
    # End for

    Post2 = Post.transpose(2,0,1)
    Post_line = Post2.T.flatten()
    Pre_line = np.zeros(np.size(Post_line), dtype=np.int16)
    k = 0
    for i in range(0, np.size(Post_line), params['N_connections']):
        Pre_line[i:(i + params['N_connections'])] = k
        k = k + 1
    return [Pre_line.flatten(), Post_line.flatten()]

### Astrocytes' synapses
The connection used in the astrocyte network is quite different from the neuron part. Each astrocyte is diffusively coupled with a maximum of **four** nearest neighbors. (${\Delta\left[Ca^{2+}\right]}^{\left(m,n\right)}$) and (${\Delta IP_3}^{\left(m,n\right)}$) are the discrete Laplace operators that can be expressed in the following equation by replacing the $\mathcal{X}$ variable:
\begin{equation}
\left(\Delta\left[\mathcal{X}\right]\right)^{\left(m,n\right)}=\left[\mathcal{X}\right]^{\left(m+1,n\right)}+\left[\mathcal{X}\right]^{\left(m-1,n\right)}+\left[\mathcal{X}\right]^{\left(m,n+1\right)}+\left[\mathcal{X}\right]^{\left(m,n-1\right)}-4\left[\mathcal{X}\right]^{\left(m,n\right)}
\end{equation}
Note that, ones which are located on the border have a lack of connectivity and instead of 4 connection, they might have 3 or even 2. That is why we have several condition in the below function.

In [4]:
def createAstrocytesConnections(params):
    # Connection of astrocytes
    # Each agent is connected to its closest neighbours (4 sides)
    Post_line = np.zeros((params['quantity_astrocytes']*4), dtype=np.int16)
    Pre_line  = np.zeros((params['quantity_astrocytes']*4), dtype=np.int16)
    n = 0
    for i in range(0, params['mastro']):
        for j in range(0, params['nastro']):
            if (i == 0 and j == 0):                                 # Corner top left
                Pre_line[n:(n+2)] = j*params['mastro'] + i
                Post_line[n]      = (j + 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i + 1)
                n += 2
            elif (i == params['mastro']-1) and (j == params['nastro']-1): # Corner bottom right
                Pre_line[n:(n+2)] = j*params['mastro'] + i
                Post_line[n]      = (j - 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i - 1)
                n += 2
            elif (i == 0) and (j == params['nastro']-1):            # Corner top right
                Pre_line[n:(n+2)] = j*params['mastro'] + i
                Post_line[n]      = (j - 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i + 1)
                n += 2
            elif (i == params['mastro']-1) and (j == 0):            # Corner bottom left
                Pre_line[n:(n+2)] = j*params['mastro'] + i
                Post_line[n]      = (j + 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i - 1)
                n += 2
            elif i == 0:                                            # First top row
                Pre_line[n:(n+3)] = j*params['mastro'] + i
                Post_line[n]      = (j - 1)*params['mastro'] + i
                Post_line[n+1]    = (j + 1)*params['mastro'] + i
                Post_line[n+2]    = j*params['mastro'] + (i + 1)
                n += 3
            elif i == params['mastro']-1:                           # Last bottom row
                Pre_line[n:(n+3)] = j*params['mastro'] + i
                Post_line[n]      = (j - 1)*params['mastro'] + i
                Post_line[n+1]    = (j + 1)*params['mastro'] + i
                Post_line[n+2]    = j*params['mastro'] + (i - 1)
                n += 3
            elif j == 0:                                            # First left column
                Pre_line[n:(n+3)] = j*params['mastro'] + i
                Post_line[n]      = (j + 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i - 1)
                Post_line[n+2]    = j*params['mastro'] + (i + 1)
                n += 3
            elif j == params['nastro']-1:                           # Last right column
                Pre_line[n:(n+3)] = j*params['mastro'] + i
                Post_line[n]      = (j - 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i - 1)
                Post_line[n+2]    = j*params['mastro'] + (i + 1)
                n += 3
            elif (i == 0 and j == 0):                               # Corner top left
                Pre_line[n:(n+2)] = j*params['mastro'] + i
                Post_line[n]      = (j + 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i + 1)
                n += 2
            elif (i == params['mastro']-1) and (j == params['nastro']-1): # Corner bottom right
                Pre_line[n:(n+2)] = j*params['mastro'] + i
                Post_line[n]      = (j - 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i - 1)
                n += 2
            elif (i == 0) and (j == params['nastro']-1):            # Corner top right
                Pre_line[n:(n+2)] = j*params['mastro'] + i
                Post_line[n]      = (j - 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i + 1)
                n += 2
            elif (i == params['mastro']-1) and (j == 0):            # Corner bottom left
                Pre_line[n:(n+2)] = j*params['mastro'] + i
                Post_line[n]      = (j + 1)*params['mastro'] + i
                Post_line[n+1]    = j*params['mastro'] + (i - 1)
                n += 2
            elif (i > 0) and (i < params['mastro']-1) and (j > 0) and (j < params['nastro']-1): # Middle nodes
                Pre_line[n:(n+4)] = j*params['mastro'] + i
                Post_line[n]      = (j - 1)*params['mastro'] + i
                Post_line[n+1]    = (j + 1)*params['mastro'] + i
                Post_line[n+2]    = j*params['mastro'] + (i - 1)
                Post_line[n+3]    = j*params['mastro'] + (i + 1)
                n += 4
    # End for
    Pre_line  = Pre_line[0:n]
    Post_line = Post_line[0:n]
    return [Pre_line.flatten(), Post_line.flatten()]

### Loading & preparing patterns
The below functions are utilized to load and also merge them with noise. `load_images` can be used for loading, and `make_experiment` for adding noise.

In [5]:
def load_images(params):
    images = np.zeros((params['mneuro'], params['nneuro'], len(params['image_names'])))
    i = 0
    for name in params['image_names']:
        image = imread(params['images_dir'] + '/' + name)
        image = cvtColor(image, COLOR_BGR2GRAY)
        images[:,:,i] = image
        i += 1
    return images

def make_experiment(images, params):
    num_images = len(images)
    # Images are affected by noise and prepared
    if 'learn_order' in params:
        learn_order = params['learn_order']
    else:
        learn_order = make_image_order(num_images, 10, True)
    learn_signals = make_noise_signals(images, \
                                            learn_order, \
                                            params['mneuro'], \
                                            params['nneuro'], \
                                            params['variance_learn'], \
                                            params['Iapp_learn'])
    if 'test_order' in params:
        test_order = params['test_order']
    else:
        test_order = make_image_order(num_images, 1, True)
    test_signals = make_noise_signals(images, \
                                            test_order, \
                                            params['mneuro'], \
                                            params['nneuro'], \
                                            params['variance_test'], \
                                            params['Iapp_test'])
    I_signals = np.concatenate((learn_signals, test_signals), axis=2)
    I_signals = np.uint8(I_signals)

    # Make image signals
    learn_timeline = make_timeline(params['learn_start_time'], \
                                   params['learn_impulse_duration'], \
                                   params['learn_impulse_shift'], \
                                   len(learn_order))
    test_timeline  = make_timeline(params['test_start_time'], \
                                   params['test_impulse_duration'], \
                                   params['test_impulse_shift'], \
                                   len(test_order))
    
    full_timeline = np.concatenate((learn_timeline, test_timeline), axis=0)
    full_timeline = np.int0(full_timeline / params['step'])
    full_timeline = np.uint16(full_timeline)
    
    timeline_signal_id       = np.zeros((params['n']), dtype=np.int8)
    timeline_signal_id_movie = np.zeros((params['n']), dtype=np.int8)
    
    for i in range(0, I_signals.shape[2]):
        be = full_timeline[i, 0]
        en = full_timeline[i, 1]
        # For simulation
        timeline_signal_id[be : en] = i + 1
        # For video
        be = be - params['before_sample_frames']
        en = en + params['after_sample_frames']
        timeline_signal_id_movie[be : en] = i + 1
    return [I_signals, full_timeline, timeline_signal_id, timeline_signal_id_movie]

def make_image_order(num_images, num_repetitions, need_shuffle):
    image_order = np.zeros((num_images * num_repetitions), dtype=np.int8)
    for id_image in range(0, num_images):
        image_order[(id_image*num_repetitions):((id_image+1)*num_repetitions)] = id_image
    if need_shuffle:
        image_order = image_order(np.random.permutation(num_images*num_repetitions))
    return image_order

def make_noise_signals(images, order, height, width, variance, Iapp0):
    signals = np.zeros((height, width, len(order)))
    for i in range(0, len(order)):
        image_id         = order[i]
        signal           = make_noise_signal(images[:,:,image_id], height, width, variance, Iapp0)
        signals[:, :, i] = signal
    return signals

def make_noise_signal(image, height, width, variance, Iapp0, thr=127):
    image = image[0 : height, 0 : width] < thr
    # rng('shuffle')
    p = np.random.permutation(width*height)
    b = p[0 : np.uint16(width * height * variance)]
    image[b] = ~image[b]
    image = np.double(image) * Iapp0
    return image

def make_timeline(start, duration, step, num_samples):
    timeline = np.zeros((num_samples, 2))
    for i in range(0, num_samples):
        be             = start + step*i
        en             = be + duration
        timeline[i, :] = [be, en]
    return timeline

### Neuron to astrocyte
This part comprises the bunch of functions defined to connect **Neuron** to **Astrocyte**.

In [6]:
def neuron2astrocyte(params, G, maskLine):
    mask = np.reshape(maskLine, (params['nneuro'], params['mneuro'])).T
    mask = np.single(mask)
    
    glutamate_above_thr = np.reshape(G, (params['nneuro'], params['mneuro'])).T
    
    neuron_astrozone_activity = np.zeros((params['mastro'], params['nastro']))
    neuron_astrozone_spikes   = np.zeros((params['mastro'], params['nastro']), dtype=np.int8)
    
    sj = int(0)
    for j in range(0, params['mneuro'], params['az']):
        sk = int(0)
        for k in range(0, params['nneuro'], params['az']):
            # The number of neurons that passed the glu threshold
            neuron_astrozone_activity[j - sj, k - sk] = \
                np.add.reduce(glutamate_above_thr[j:(j+2), k:(k+2)], axis=(0,1))
                # np.sum(glutamate_above_thr[j:(j+2), k:(k+2)])  # The 'reduce' order is faster
            # Number of neurons spiking
            neuron_astrozone_spikes[j - sj, k - sk] = \
                np.add.reduce(mask[j:(j+2), k:(k+2)], axis=(0,1))
                # np.sum(mask[j:(j+2), k:(k+2)])  # The 'reduce' order is faster
            sk = int((k + 2)/2)
        sj = int((j + 2)/2)
    return neuron_astrozone_activity.T.flatten(), neuron_astrozone_spikes.T.flatten()

## Valuation
This section provides the privilege to define **constant** values, which are usually used with a unique value in the whole project. Note that, they must be defined as a part of the `params` variable, which is a *list*.

In [7]:
params = {}
# The below variables are time-step and simulation time, respectively.
# You might need to alter them arbitrarily.
params['step'] = 0.0001 #(double)
params['tOut'] = 6      #(double)

# The given line below is a dependent variable. You normally
# should NOT change it.
params['n'] = int(params['tOut']/params['step']) #(DEPENDENT)

# The two next variables carry the folders from which you can use to
# call your files and saving the results organizely.
params['loadPath'] = 'data/inputs'  #(string)
params['savePath'] = 'data/outputs' #(string)

# Do you want to save a diary after each simulation? So set the below logical
# variable "True". The below directory is the place your logs are saved.
# The third order is a string that carries the name of the file, and normally
# is created using the current time, but you can change arbitrary.
params['makeDiary'] = True   #(logical)
params['diaryDir']  = 'logs' #(string)
params['diaryFile'] = lib['func'].getNow(1, '-') #(string)

# The amount of time (in second) between each commands printed on CW.
params['commandIntervalSpan'] = 2 #(int)

# The below line is related to creating names when you are saving data
# which include the time of saving, It is a logical variable which has
# "False" value as its default
params['uniqueSave'] = False #(logical)

# The below string can be set one of the following formats. It's the
# default format whcih when you do not insert any formats, it will be
# considered. These legall formats are "jpg", "png", "pdf"
params['defaultImageFormat'] = 'png' #(string)

#### Your code ------------------------------------------------------
# Load Images options
params['images_dir'] = params['loadPath'] + '/images - patterns/Experiment 1 - chars 10'
params['image_names'] = [
    'A.jpg',        \
    'mid_B.jpg',    \
    'P.jpg',        \
    'mid_L.jpg',    \
    'J.jpg',        \
    'I.jpg',        \
    'E.jpg',        \
    'G.jpg'         \
]

# Experiment
params['learn_start_time']         = 0.2
params['learn_impulse_duration']   = 0.21
params['learn_impulse_shift']      = 0.41
params['learn_order']              = np.array([0, 1, 2, 3])

params['test_start_time']          = 2.3
params['test_impulse_duration']    = 0.13 # 0.15
params['test_impulse_shift']       = 0.42 # 0.4
params['test_order']               = np.array([0, 4, 1, 5, 2, 6, 3, 7])

# Applied pattern current
params['variance_learn']           = 0     # 0.05
params['variance_test']            = 0     # 0.03
params['Iapp_learn']               = 10    # 80
params['Iapp_test']                = 8     # 8

# Movie
params['after_sample_frames']      = 200
params['before_sample_frames']     = 1

# Poisson noise
params['poisson_nu']                = 1.5
params['poisson_n_impulses']        = 15
params['poisson_impulse_duration']  = int(0.03 / params['step'])
params['poisson_impulse_initphase'] = int(1.5 / params['step'])
params['poisson_amplitude']         = 20

# Network size
# (mneuro = mastro * 3 + 1 (for az = 4))
params['mneuro']                   = 10
params['nneuro']                   = 10
params['quantity_neurons']         = params['mneuro'] * params['nneuro']
params['mastro']                   = 5
params['nastro']                   = 5
params['quantity_astrocytes']      = params['mastro'] * params['nastro']
params['az']                       = 2

# Neuron model
params['alf']                      = 10   # s^-1    | Glutamate clearance constant
params['k']                        = 600  # uM.s^-1 | Efficacy of glutamate release

# Synaptic connections
params['N_connections']            = 20   #number of synapses per neurons 
params['quantity_connections']     = params['quantity_neurons'] * params['N_connections']
params['lambdagain']               = 1.5  #Average exponential distribution

params['enter_astro']              = 3    #F_astro # F_recall
params['F_memorize']               = 0.6  #F_act   # F_memorize
params['t_neuro']                  = 0.06          # Astrocyte effect duration (second)
params['amplitude_neuro']          = 5             # Astrocyte input
params['threshold_Ca']             = 0.3  #0.15    # calcium should be higher than this to have WM

window_astro_watch                 = 0.01          # t(sec) # astrocyte watching this much back to neurons with bnh intervals
shift_window_astro_watch           = 0.001         # t(sec)
impact_astro                       = 0.26 #0.25    # t(sec)
params['impact_astro']             = int(impact_astro / params['step'])
params['window_astro_watch']       = int(window_astro_watch / params['step'])
params['shift_window_astro_watch'] = int(shift_window_astro_watch / params['step'])

## Signals
Any signals and array variables should be defined in this section. Also, scope objects which are effective to observe a signal, should be defined here. All your signals must be a part of `signals` variable.

In [8]:
signals = {}
# Simulation time vector
signals['tLine'] = np.arange(0, params['tOut'], params['step'])

# Put your signals here ~~~>
#  Prepare images
signals['images'] = load_images(params)
signals['Iapp'], signals['T_Iapp'], signals['T_Iapp_met'], \
    signals['T_record_met'] = make_experiment(signals['images'], params)

signals['neuronsPre'], signals['neuronsPost']       = createNeuronsConnections(params)
signals['astrocytesPre'], signals['astrocytesPost'] = createAstrocytesConnections(params)

## Models
Any dynamic objects and those which are not as simple as an array must be defined as a part of `models` variable; Objects such as estimators and controllers.

In [9]:
models = {}
# This function is created to support your systems
models['updated'] = True
# Put your signals here ~~~>

## Simulation
The key function of the project, undoubtedly, can be mentioned is `simulation` sunction which is given below. In this part, regarding the availability of all variables (`params`, `signals`, `models`, and `lib`) you are able to code your main purpose of this project here. Note that there is a loop named `Main loop` which you can utilize to use as time step loop, although you might not need to use that in many projects.

In [10]:
def simulation(params, signals, models, lib):
    # [Parameters, Models, Signals, Libraries] <- (Parameters, Signals, Models, Libraries)
    # This function is your main code which there is a time-loop in
    # Also, the model blocks and the signals must be updated here.
    # Before the main loop, you can initialize if you need, Also you can
    # finalize after that if you need, as well.
    
    ## Initial options
    func = lib['func']
    st   = func.sayStart(params)
    # A trigger used to report steps in command
    trig = [st, params['commandIntervalSpan'], -1]
    
    ## Main loop
    for k in range(0, params['n']):
        # Displaying the iteration number on the command window
        trig = func.disit(k, params['n'], trig, params)

        # Put your signals here ~~~>

    ## Finalize options
    # To report the simulation time after running has finished
    func.disit(k, params['n'], [st, 0, trig[2]], params)
    func.sayEnd(st, params)
    # Sent to output
    return [params, signals, models]

## Running
To run the project, the below piece of code is provided. In simple projects, you might need to run once, while there are numerous ones that you will have to run the simulation for several times; to do that, you can use loops and other arbitrary techniques to call `simulation` which might be given changed inputs such as `params`, etc.

In [ ]:
[params, signals, models] = simulation(params, signals, models, lib)

### Profiling
If you need to check the timing of your project and know which part of your program is the most time-consuming, uncomment the content in this section and observe your performance of coding. It is worth mentioning that the previous code block should be commented.

In [ ]:
# # Running the code and save the results into a file
# order = 'simulation(params, signals, models, lib)'
# profileName = params['diaryDir'] + '/profiler/' + params['diaryFile'] + '.prof'
# cProfile.run(order, profileName)
# # Depiction of the results 
# %load_ext snakeviz
# %snakeviz profileName

## Illustration
If you need to demonsrate the results obtained, this section can be utilized to have better organization.

In [12]:
## Initialize
draw  = lib['draw']            # Import all plotting functions
n     = params['n']
nn    = np.arange(0, n)        # A vector from 1 to n
tLine = signals['tLine'][nn]   # A time-line vector
## Write your codes here ~~~>

## Saving
Use the below part, providing to save data.

In [13]:
## Write your codes here ~~~>